<a href="https://colab.research.google.com/github/Dominicpham03/CarPricePrediction/blob/main/Final_Project_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ACI-IoT-2023.csv')

In [ ]:

#Remove dup and missing value
df.drop_duplicates()
df.dropna()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Connection Type
0,192.168.1.7-54.230.163.60-33344-443-6,192.168.1.7,33344,54.230.163.60,443,6,2023-01-11 09:43:40,379933,11,11,...,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
1,192.168.3.6-91.189.91.48-36754-80-6,192.168.3.6,36754,91.189.91.48,80,6,2023-01-11 09:43:51,205637,3,3,...,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
3,192.168.1.9-192.168.1.1-8080-40426-6,192.168.1.9,8080,192.168.1.1,40426,6,2023-01-11 09:43:49,5030379,1,2,...,0.000000e+00,0.000000e+00,0.0,0.0,8.494231e+14,1.201266e+15,1.698846e+15,5.030379e+06,Benign,wired
4,192.168.1.20-35.232.111.17-40054-80-6,192.168.1.20,40054,35.232.111.17,80,6,2023-01-11 09:43:56,72278,3,4,...,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
5,192.168.1.20-35.232.111.17-40054-80-6,192.168.1.20,40054,35.232.111.17,80,6,2023-01-11 09:43:56,2457,2,0,...,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231406,192.168.1.24-20.189.173.11-58126-443-6,192.168.1.24,58126,20.189.173.11,443,6,2023-02-11 13:27:25,1,2,0,...,0.000000e+00,0.000000e+00,0.0,0.0,1.698946e+15,0.000000e+00,1.698946e+15,1.698946e+15,Benign,wired
1231407,209.206.49.216-192.168.1.87-30001-47579-6,209.206.49.216,30001,192.168.1.87,47579,6,2023-02-11 13:27:20,50077229,4,2,...,7.708200e+04,0.000000e+00,77082.0,77082.0,8.494730e+14,1.201336e+15,1.698946e+15,4.992308e+07,Benign,wired
1231408,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2023-02-11 13:27:26,46365348,15,0,...,3.713166e+06,3.789721e+06,8045778.0,1014561.0,4.247365e+14,8.494730e+14,1.698946e+15,6.831749e+06,Benign,wired
1231409,192.168.1.247-52.123.129.14-52046-443-6,192.168.1.247,52046,52.123.129.14,443,6,2023-02-11 13:27:36,146262,15,37,...,0.000000e+00,0.000000e+00,0.0,0.0,1.698946e+15,0.000000e+00,1.698946e+15,1.698946e+15,Benign,wired


In [ ]:

pd.set_option('display.max_columns', None)
df.describe(include='all')



,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Connection Type
count,1231411,1231411,1.231411e+06,1231411,1.231411e+06,1.231411e+06,1231411,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.230402e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1231411.0,1.231411e+06,1231411.0,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1231411.0,1231411.0,1231411.0,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1231411.0,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1.231411e+06,1231411,1231411
unique,865406,159381,NaN,89089,NaN,NaN,75403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,2
top,8.6.0.1-8.0.6.4-0-0-0,192.168.1.45,NaN,192.168.1.1,NaN,NaN,2023-10-30 12:21:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Port Scan,wireless
freq,3685,621729,NaN,255771,NaN,NaN,16208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441282,742758
mean,NaN,NaN,3.680030e+04,NaN,1.540297e+04,6.433792e+00,NaN,5.236489e+06,5.456022e+00,5.439780e+00,4.998453e+03,2.078230e+04,1.060881e+02,1.164441e+01,3.245995e+01,3.525900e+01,1.514058e+02,1.412901e+01,4.599782e+01,4.432950e+01,inf,inf,2.566652e+06,6.524624e+05,3.608930e+06,2.173028e+06,5.162231e+06,2.980284e+06,3.911888e+05,3.566219e+06,2.735757e+06,2.179595e+06,8.974230e+05,1.619241e+05,1.175485e+06,7.880951e+05,5.276873e-02,0.0,3.069649e-04,0.0,1.462479e+02,1.524636e+02,2.172460e+0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231411 entries, 0 to 1231410
Data columns (total 85 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Flow ID                     1231411 non-null  object 
 1   Src IP                      1231411 non-null  object 
 2   Src Port                    1231411 non-null  int64  
 3   Dst IP                      1231411 non-null  object 
 4   Dst Port                    1231411 non-null  int64  
 5   Protocol                    1231411 non-null  int64  
 6   Timestamp                   1231411 non-null  object 
 7   Flow Duration               1231411 non-null  int64  
 8   Total Fwd Packet            1231411 non-null  int64  
 9   Total Bwd packets           1231411 non-null  int64  
 10  Total Length of Fwd Packet  1231411 non-null  float64
 11  Total Length of Bwd Packet  1231411 non-null  float64
 12  Fwd Packet Length Max       1231411 non-null  float64
 1

In [ ]:
features = df.columns.tolist()




In [ ]:
features.remove("Flow ID")
features.remove("Src IP")
features.remove("Dst IP")






In [ ]:
X = df[features]
y = df["Label"]
unique_labels = np.unique(y)


data = pd.concat([X, y], axis=1)
print(unique_labels)

['ARP Spoofing' 'Benign' 'DNS Flood' 'Dictionary Attack' 'ICMP Flood'
 'OS Scan' 'Ping Sweep' 'Port Scan' 'SYN Flood' 'Slowloris' 'UDP Flood'
 'Vulnerability Scan']


In [ ]:









# Keep 0.008% of the data
data_reduced = data.sample(frac=0.01, random_state=42)

# Re-shuffle
data_reduced = data_reduced.sample(frac=1.0, random_state=42).reset_index(drop=True)
data_reduced = data_reduced.loc[:, ~data_reduced.columns.duplicated()]

# Separate features and labels
X_reduced = data_reduced[features]

y_reduced = data_reduced["Label"]


if "Label" in X.columns:
    X_reduced = X_reduced.drop("Label", axis=1)

print(X_reduced.shape)
print(y_reduced.shape)
print(type(y_reduced))
print(y_reduced.head())

# Label encode each column and store as a DataFrame
def encode_column(colname):
    le = LabelEncoder()
    col = X_reduced[colname]
    return pd.Series(le.fit_transform(col), name=colname)


X_encoded = pd.concat([encode_column(col) for col in X_reduced.columns], axis=1)


(12314, 81)
(12314,)
<class 'pandas.core.series.Series'>
0        Benign
1    ICMP Flood
2        Benign
3        Benign
4    ICMP Flood
Name: Label, dtype: object


In [ ]:

distinct_labels = list(y_reduced.unique())
print(distinct_labels)



['Benign', 'ICMP Flood', 'Port Scan', 'DNS Flood', 'Ping Sweep', 'Vulnerability Scan', 'Slowloris', 'OS Scan', 'SYN Flood', 'Dictionary Attack', 'UDP Flood']


In [ ]:
# from joblib import Parallel, delayed


# def encode_column(col):
#     le = LabelEncoder()
#     return le.fit_transform(col)

# # Example for a DataFrame with multiple columns to encode
# df_encoded = pd.DataFrame(
#     Parallel(n_jobs=-1)(delayed(encode_column)(df[col]) for col in df.columns),
#     index=df.index
# )

In [ ]:
from imblearn.over_sampling import SMOTE

X_train ,X_test, y_train, y_test = train_test_split(X_encoded,y_reduced,test_size = 0.2,random_state=68)

smote = SMOTE(k_neighbors=1, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

le = LabelEncoder()
y_resampled = le.fit_transform(y_resampled)
y_test = le.transform(y_test)

In [ ]:

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

clf1 = RandomForestClassifier(random_state=42, class_weight='balanced')
clf2 = xgb.XGBClassifier(random_state=42,class_weight='balanced')




In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint



#define parameter range
param_dist = {
    'n_estimators': randint(100, 200),
    'max_depth': randint(10, 30),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': [ 'sqrt', 'log2'],
}

# Perform RandomizedSearchCV for RandomForest
random_search = RandomizedSearchCV(
    estimator=clf1,
    param_distributions=param_dist,
    n_iter=50,
    cv=3,
    verbose=2,
    n_jobs=-1,  # Use all cores
    random_state=42
)


random_search.fit(X_resampled, y_resampled)


print("Best parameters:", random_search.best_params_)

# Get the best RandomForest model
best_rf = random_search.best_estimator_



Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters: {'max_depth': 22, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 170}


In [ ]:
clf1 = RandomForestClassifier(
    max_depth=22,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=170,
    random_state=42,
    n_jobs=-1
)

In [ ]:


clf1.fit(X_resampled,y_resampled)






RandomForestClassifier(max_depth=22, n_estimators=170, n_jobs=-1,
                       random_state=42)

In [ ]:
clf2.fit(X_resampled,y_resampled)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:56:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

ensemble = StackingClassifier(
    estimators=[
        ('clf1', clf1),
        ('clf2', clf2)
    ],
    final_estimator=LogisticRegression()
)

ensemble.fit(X_resampled, y_resampled)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:57:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:57:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:57:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:58:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:58:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight" } are not used.



StackingClassifier(estimators=[('clf1',
                                RandomForestClassifier(max_depth=22,
                                                       n_estimators=170,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('clf2',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              class_weight='balanced',
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metr...
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None, ...))],
                   final_estimator=LogisticRegression())

In [ ]:

y_pred = ensemble.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9947
